In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px  # for maps

%matplotlib inline

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (11, 6)
pd.set_option("display.max_columns", 100)

results_path = "data/raw/epi2024results.csv"
variables_path = "data/processed/epi2024variables2024-12-11.csv"

df_raw = pd.read_csv(results_path)
print("Results shape:", df_raw.shape)
display(df_raw.head())

var_meta = pd.read_csv(variables_path)
print("Variables metadata shape:", var_meta.shape)
display(var_meta.head())

id_cols = [col for col in ["country", "iso", "code"] if col in df_raw.columns]
new_cols = [col for col in df_raw.columns if col.endswith(".new")]

df = df_raw[id_cols + new_cols].copy()
print("Selected columns:", len(df.columns))
display(df.head())

print("\n--- df.info() ---")
df.info()

print("\n--- Missing values per column (non-zero only) ---")
missing = df.isna().sum().sort_values(ascending=False)
display(missing[missing > 0])

na_ratio = df.isna().mean()
high_na = na_ratio[na_ratio > 0.50].index.tolist()
print("\nIndicators dropped due to high missing ratio (>50%):", len(high_na))

df_clean = df.drop(columns=high_na).copy()
print("Shape after dropping high-NA columns:", df_clean.shape)

num_cols = df_clean.select_dtypes(include=[np.number]).columns
df_clean[num_cols] = df_clean[num_cols].fillna(df_clean[num_cols].median())

print("Total remaining missing values:", df_clean.isna().sum().sum())

print("\n--- Descriptive statistics (first 10 rows) ---")
desc = df_clean.describe().T
display(desc.head(10))

var_meta_short = var_meta[[
    "Abbreviation", "Variable", "Type",
    "IssueCategory", "PolicyObjective", "Weight", "Description"
]].copy()

var_meta_short = var_meta_short.dropna(subset=["Abbreviation"]).drop_duplicates(subset=["Abbreviation"])

indicator_cols = [c for c in df_clean.columns if c.endswith(".new")]
base_names = [c.replace(".new", "") for c in indicator_cols]

indicator_df = pd.DataFrame({
    "column_name": indicator_cols,
    "Abbreviation": base_names
})

indicator_with_meta = indicator_df.merge(var_meta_short, on="Abbreviation", how="left")

print("\n--- Indicators with metadata (first 15) ---")
display(indicator_with_meta.head(15))

candidate_abbr = ["EPI", "ECO", "BDH", "AIR", "WSH", "GHN", "CBP", "CLP"]
key_cols = []
for abbr in candidate_abbr:
    col_name = abbr + ".new"
    if col_name in df_clean.columns:
        key_cols.append(col_name)

print("\nKey indicator columns found:", key_cols)
if key_cols:
    display(df_clean[key_cols].describe().T)

if "EPI.new" in df_clean.columns:
    plt.figure()
    sns.histplot(df_clean["EPI.new"], kde=True, bins=25)
    plt.title("Distribution of Overall Environmental Performance (EPI.new)")
    plt.xlabel("EPI Score")
    plt.ylabel("Number of Countries")
    plt.show()
else:
    print("EPI.new not found in df_clean.")

if "EPI.new" in df_clean.columns:
    df_sorted = df_clean.sort_values("EPI.new", ascending=False)
    top10 = df_sorted.head(10)
    bottom10 = df_sorted.tail(10)

    print("\n--- Top 10 countries by EPI.new ---")
    display(top10[["country", "EPI.new"]])

    print("\n--- Bottom 10 countries by EPI.new ---")
    display(bottom10[["country", "EPI.new"]])

    plt.figure()
    sns.barplot(data=top10, x="EPI.new", y="country")
    plt.title("Top 10 Countries by EPI Score")
    plt.xlabel("EPI Score")
    plt.ylabel("Country")
    plt.xlim(0, 100)
    plt.show()

    plt.figure()
    sns.barplot(data=bottom10, x="EPI.new", y="country")
    plt.title("Bottom 10 Countries by EPI Score")
    plt.xlabel("EPI Score")
    plt.ylabel("Country")
    plt.xlim(0, 100)
    plt.show()

for col in key_cols:
    plt.figure()
    sns.histplot(df_clean[col], kde=True, bins=25)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Number of Countries")
    plt.show()

if key_cols:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df_clean[key_cols])
    plt.title("Boxplots of Selected Environmental Indicators")
    plt.xticks(rotation=45)
    plt.show()

num_df = df_clean.select_dtypes(include=[np.number])

corr = num_df.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr, cmap="coolwarm", center=0, linewidths=0.4)
plt.title("Correlation Matrix of Environmental Indicators")
plt.show()

if "EPI.new" in corr.columns:
    corr_with_epi = corr["EPI.new"].sort_values(ascending=False)
    print("\n--- Correlation of indicators with EPI.new (top 15) ---")
    display(corr_with_epi.head(15))

if "GHN.new" in df_clean.columns and "EPI.new" in df_clean.columns:
    plt.figure()
    sns.scatterplot(data=df_clean, x="GHN.new", y="EPI.new")
    plt.title("EPI.new vs GHN.new")
    plt.xlabel("GHN.new (Greenhouse Gas–related Score)")
    plt.ylabel("EPI.new")
    plt.show()

if "AIR.new" in df_clean.columns and "EPI.new" in df_clean.columns:
    plt.figure()
    sns.scatterplot(data=df_clean, x="AIR.new", y="EPI.new")
    plt.title("EPI.new vs AIR.new")
    plt.xlabel("AIR.new (Air Quality Score)")
    plt.ylabel("EPI.new")
    plt.show()

if "WSH.new" in df_clean.columns and "EPI.new" in df_clean.columns:
    plt.figure()
    sns.scatterplot(data=df_clean, x="WSH.new", y="EPI.new")
    plt.title("EPI.new vs WSH.new")
    plt.xlabel("WSH.new (Water, Sanitation & Hygiene)")
    plt.ylabel("EPI.new")
    plt.show()

if all(col in df_clean.columns for col in ["iso", "country", "EPI.new"]):
    map_df = df_clean.dropna(subset=["iso", "EPI.new"]).copy()

    fig = px.choropleth(
        map_df,
        locations="iso",
        color="EPI.new",
        hover_name="country",
        color_continuous_scale="Viridis",
        range_color=(map_df["EPI.new"].min(), map_df["EPI.new"].max()),
        title="Environmental Performance Index (EPI.new) – 2024 Global Map"
    )

    fig.update_layout(
        geo=dict(showframe=False, showcoastlines=True),
        coloraxis_colorbar=dict(title="EPI Score")
    )

    fig.show()
else:
    print("Cannot draw map: iso, country, or EPI.new missing.")

def plot_indicator_map(df, col, title, colorbar_title):
    temp = df.dropna(subset=["iso", col]).copy()
    fig = px.choropleth(
        temp,
        locations="iso",
        color=col,
        hover_name="country",
        color_continuous_scale="Viridis",
        range_color=(temp[col].min(), temp[col].max()),
        title=title,
    )
    fig.update_layout(
        geo=dict(showframe=False, showcoastlines=True),
        coloraxis_colorbar=dict(title=colorbar_title)
    )
    fig.show()

if "AIR.new" in df_clean.columns:
    plot_indicator_map(
        df_clean,
        col="AIR.new",
        title="Air Quality Score (AIR.new) – 2024",
        colorbar_title="Air Quality"
    )

if "GHN.new" in df_clean.columns:
    plot_indicator_map(
        df_clean,
        col="GHN.new",
        title="Greenhouse Gas Score (GHN.new) – 2024",
        colorbar_title="GHN Score"
    )

if "CBP.new" in df_clean.columns:
    plot_indicator_map(
        df_clean,
        col="CBP.new",
        title="Carbon Balance Progress (CBP.new) – 2024",
        colorbar_title="CBP Score"
    )

print("\nEDA completed.")
